# Example notebook training of an RL agent using Ray RLlib
#### *(using th CartPole-v1 gym environnment)*

### Import requirements

In [1]:
import ray
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.models import ModelCatalog
import numpy as np
import matplotlib.pyplot as plt
import LTCRL.utils as lru               # Utilities for training LTCs with pytorch
import LTCRL.models as models           # RayRLlib model implementations
from ray.rllib.examples.models.rnn_model import TorchRNNModel
ray.init()

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:45652',
 'object_store_address': '/tmp/ray/session_2022-03-14_14-14-37_049828_59049/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-14_14-14-37_049828_59049/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-03-14_14-14-37_049828_59049',
 'metrics_export_port': 62205,
 'gcs_address': '127.0.0.1:57162',
 'node_id': '0f3300e6b52dfa46dcb8206bd8f8470a3716f79a5dfc79137fc9a150'}

### Define a config dictionary for our trainer 

In [5]:
ModelCatalog.register_custom_model("naiveRNN", models.NaiveRNN)

config = {
    # Environment (RLlib understands openAI gym registered strings).
    "env": "CartPole-v1",
    # Use 2 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 1,
    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "torch",
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    "model": {
        "custom_model": "naiveRNN",
        "max_seq_len": 20,
        "custom_model_config": {
            "cell_size": 32,
        },
    },
    # Set up a separate evaluation worker set for the
    # `trainer.evaluate()` call after training (see below).
    "evaluation_num_workers": 1,
    # Only for evaluation runs, render the env.
    "evaluation_config": {
        "render_env": True,
    }
}

### Train the agent with our defined config

In [8]:
# Instanciate the PPO trainer object
trainer = PPOTrainer(config=config)

# Run it for n training iterations. A training iteration includes
# parallel sample collection by the environment workers as well as
# loss calculation on the collected batch and a model update.
log = []
iterations = 1
for i in range(iterations):
    print("iteration : " +str(i), ", ")
    log.append(trainer.train())
    print('len : ' + str(log[i]['episode_len_mean']))
    print('avg_rev : ' + str(np.array(log[i]['hist_stats']['episode_reward']).mean()))
    # if i % 5 == 0:
        # trainer.evaluate()
        # trainer.evaluate()

2022-03-14 14:13:47,994	WARNING trainer.py:2279 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Trainer.train()`. Instead, you will have to call `Trainer.evaluate()` manually in order to trigger an evaluation run.
(pid=59007) 
(RolloutWorker pid=59007) 2022-03-14 14:13:53,669	WARNING catalog.py:544 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
(RolloutWorker pid=59007) 2022-03-14 14:13:53,682	ERROR worker.py:432 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=59007, ip=127.0.0.1)
(RolloutWorker pid=59007)   File "/Users/renard/miniconda3/envs/torchLTC/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 588, in __init__
(RolloutWorker pid=59007)     self._build_policy_map(
(RolloutWorker pid=59007

RayActorError: The actor died because of an error raised in its creation task, [36mray::RolloutWorker.__init__()[39m (pid=59007, ip=127.0.0.1)
  File "/Users/renard/miniconda3/envs/torchLTC/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 588, in __init__
    self._build_policy_map(
  File "/Users/renard/miniconda3/envs/torchLTC/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1555, in _build_policy_map
    self.policy_map.create_policy(name, orig_cls, obs_space, act_space,
  File "/Users/renard/miniconda3/envs/torchLTC/lib/python3.8/site-packages/ray/rllib/policy/policy_map.py", line 143, in create_policy
    self[policy_id] = class_(observation_space, action_space,
  File "/Users/renard/miniconda3/envs/torchLTC/lib/python3.8/site-packages/ray/rllib/agents/ppo/ppo_torch_policy.py", line 50, in __init__
    self._initialize_loss_from_dummy_batch()
  File "/Users/renard/miniconda3/envs/torchLTC/lib/python3.8/site-packages/ray/rllib/pol

In [7]:
mean_rev = np.array(list(l['episode_reward_mean'] for l in log))
plt.plot(mean_rev)
plt.title('Mean reward (reward = ep_length)')

NameError: name 'log' is not defined

In [10]:
# Evaluate the trained Trainer (and render each timestep to the shell's
# output).
trainer.evaluate()

KeyboardInterrupt: 